In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import koreanize_matplotlib
from datetime import datetime, timedelta
import time
import dateutil
from pathlib import Path

In [63]:
!pip install koreanize-matplotlib

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
PATH = r"C:\Users\kmfm1\BIproject\data"

In [ ]:
customer_feedback = pd.read_csv(PATH + r"\customer_feedback.csv")
customers = pd.read_csv(PATH + r"\customers.csv")
delivery_performance = pd.read_csv(PATH + r"\delivery_performance.csv")
inventory = pd.read_csv(PATH + r"\inventory.csv")
inventoryNew = pd.read_csv(PATH + r"\inventoryNew.csv")
order_items = pd.read_csv(PATH + r"\order_items.csv")
orders = pd.read_csv(PATH + r"\Business.csv")
products = pd.read_csv(PATH + r"\products.csv")
marketing_performance = pd.read_csv(PATH + r"\marketing_performance.csv")
Business = pd.read_csv(PATH + r"\Business.csv")

In [66]:
# 1) inventory: 일 -> 월(YYYY-MM) 집계
inv_dt = pd.to_datetime(inventory["date"], dayfirst=True, errors="coerce")
inv_month = (
    inventory.assign(date=inv_dt.dt.to_period("M").astype(str))
    .groupby(["product_id", "date"], as_index=False)[["stock_received", "damaged_stock"]]
    .sum()
)
inv_month

,product_id,date,stock_received,damaged_stock
0,4452,2023-03,6,10
1,4452,2023-04,24,18
2,4452,2023-05,33,8
3,4452,2023-06,48,20
4,4452,2023-07,42,14
...,...,...,...,...
5613,993331,2024-07,17,14
5614,993331,2024-08,26,18
5615,993331,2024-09,44,14
5616,993331,2024-10,20,18


In [67]:
# 2) inventoryNew: 월 라벨 -> 월(YYYY-MM) 파싱 후 월 집계(중복 제거 효과)
invn_dt = pd.to_datetime(inventoryNew["date"], format="%b-%y", errors="coerce")
invn_month = (
    inventoryNew.assign(date=invn_dt.dt.to_period("M").astype(str))
    .groupby(["product_id", "date"], as_index=False)[["stock_received", "damaged_stock"]]
    .sum()
)
invn_month

,product_id,date,stock_received,damaged_stock
0,4452,2023-03,1,0
1,4452,2023-04,2,0
2,4452,2023-05,2,0
3,4452,2023-06,2,0
4,4452,2023-07,2,0
...,...,...,...,...
5566,993331,2024-07,2,0
5567,993331,2024-08,3,0
5568,993331,2024-09,8,0
5569,993331,2024-10,4,0


In [68]:
# 3) (검증용) 비교 테이블
wide = inv_month.merge(invn_month, on=["product_id", "date"], how="outer", suffixes=("_inv", "_invNew"))
wide

,product_id,date,stock_received_inv,damaged_stock_inv,stock_received_invNew,damaged_stock_invNew
0,4452,2023-03,6,10,1.0,0.0
1,4452,2023-04,24,18,2.0,0.0
2,4452,2023-05,33,8,2.0,0.0
3,4452,2023-06,48,20,2.0,0.0
4,4452,2023-07,42,14,2.0,0.0
...,...,...,...,...,...,...
5613,993331,2024-07,17,14,2.0,0.0
5614,993331,2024-08,26,18,3.0,0.0
5615,993331,2024-09,44,14,8.0,0.0
5616,993331,2024-10,20,18,4.0,0.0


In [69]:
# 4) "하나의 테이블" 최종값: inventoryNew 우선, 없으면 inventory
final = wide.copy()
for c in ["stock_received", "damaged_stock"]:
    final[c] = final[f"{c}_invNew"].where(final[f"{c}_invNew"].notna(), final[f"{c}_inv"])

final["source"] = np.where(
    final["stock_received_invNew"].notna() | final["damaged_stock_invNew"].notna(),
    "inventoryNew",
    "inventory",
)

In [70]:
# 5) products 붙여서 최종 테이블 완성
stock = final.merge(products, on="product_id", how="left")
stock.head()

,product_id,date,stock_received_inv,damaged_stock_inv,stock_received_invNew,damaged_stock_invNew,stock_received,damaged_stock,source,product_name,category,brand,price,mrp,margin_percentage,shelf_life_days,min_stock_level,max_stock_level
0,4452,2023-03,6,10,1.0,0.0,1.0,0.0,inventoryNew,Baby Wipes,Baby Care,Morar-Mistry,169.88,242.69,30.0,365,27,100
1,4452,2023-04,24,18,2.0,0.0,2.0,0.0,inventoryNew,Baby Wipes,Baby Care,Morar-Mistry,169.88,242.69,30.0,365,27,100
2,4452,2023-05,33,8,2.0,0.0,2.0,0.0,inventoryNew,Baby Wipes,Baby Care,Morar-Mistry,169.88,242.69,30.0,365,27,100
3,4452,2023-06,48,20,2.0,0.0,2.0,0.0,inventoryNew,Baby Wipes,Baby Care,Morar-Mistry,169.88,242.69,30.0,365,27,100
4,4452,2023-07,42,14,2.0,0.0,2.0,0.0,inventoryNew,Baby Wipes,Baby Care,Morar-Mistry,169.88,242.69,30.0,365,27,100


In [88]:
# stock.drop(columns=['stock_received', 'damaged_stock','source'], inplace=True)
stock.tail()

,product_id,date,stock_received_inv,damaged_stock_inv,stock_received_invNew,damaged_stock_invNew,product_name,category,brand,price,mrp,margin_percentage,shelf_life_days,min_stock_level,max_stock_level
5613,993331,2024-07,17,14,2.0,0.0,Pulses,Grocery & Staples,"Parmer, Gola and Yogi",102.44,120.52,15.0,365,28,86
5614,993331,2024-08,26,18,3.0,0.0,Pulses,Grocery & Staples,"Parmer, Gola and Yogi",102.44,120.52,15.0,365,28,86
5615,993331,2024-09,44,14,8.0,0.0,Pulses,Grocery & Staples,"Parmer, Gola and Yogi",102.44,120.52,15.0,365,28,86
5616,993331,2024-10,20,18,4.0,0.0,Pulses,Grocery & Staples,"Parmer, Gola and Yogi",102.44,120.52,15.0,365,28,86
5617,993331,2024-11,4,2,1.0,0.0,Pulses,Grocery & Staples,"Parmer, Gola and Yogi",102.44,120.52,15.0,365,28,86


- 제품 위험군 : < min_stock_level
- 품절 : 

damaged_stock은 "그 달에 받은 것 중 손상"이 아니라 "그 달에 손상 처리된 수량"

In [72]:
stock[stock["damaged_stock_inv"] > stock["stock_received_inv"]].shape

(696, 15)

In [78]:
inventory.head()

,product_id,date,stock_received,damaged_stock
0,153019,17-03-2023,4,2
1,848226,17-03-2023,4,2
2,965755,17-03-2023,1,0
3,39154,17-03-2023,4,0
4,34186,17-03-2023,3,2


In [81]:
orders.sort_values('order_date')

,order_id,product_id,quantity,unit_price,customer_id,order_date,order_total,product_name,category,brand,...,feedback_id,rating,feedback_category,sentiment,feedback_date,area,registration_date,customer_segment,total_orders,avg_order_value
575,7164827734,9436,2,88.43,9195970,2023-03-16 08:10:44,1809.72,Toothpaste,Personal Care,Naidu PLC,...,9187416,3,App Experience,Neutral,2023-03-16,Karawal Nagar,2024-06-21,New,14,517.58
3770,4028266143,937542,3,97.04,19216141,2023-03-16 08:31:08,616.45,Wheat Flour,Grocery & Staples,Khurana and Sons,...,7019941,1,Delivery,Negative,2023-03-16,Giridih,2023-05-14,Premium,9,1428.54
1017,4376524554,812607,2,416.65,72576730,2023-03-16 09:09:00,2512.22,Potatoes,Fruits & Vegetables,Rege Group,...,4933025,3,Customer Service,Neutral,2023-03-16,Chennai,2024-02-17,Regular,5,1971.67
1199,208989475,432617,2,863.18,94713625,2023-03-16 11:38:32,2973.06,Baby Wipes,Baby Care,Dora-Pillai,...,6489502,5,Delivery,Positive,2023-03-16,Barasat,2023-08-15,Premium,10,1791.16
4770,757816226,14145,3,630.84,22571994,2023-03-16 14:07:42,1308.05,Spinach,Fruits & Vegetables,"Thakur, Shah and Pingle",...,6062833,2,Customer Service,Negative,2023-03-16,Bihar Sharif,2024-07-08,Regular,2,912.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2379,2544173358,487931,3,709.08,43295656,2024-11-03 16:41:57,3448.92,Sugar,Grocery & Staples,"Bahl, Jaggi and Nath",...,6629918,4,Customer Service,Neutral,2024-11-03,Sagar,2024-02-21,Regular,4,1306.25
3834,2272849018,871592,1,870.29,6982488,2024-11-04 00:15:51,1797.17,Carrots,Fruits & Vegetables,"Shankar, Nadkarni and Kumer",...,4333600,3,Delivery,Neutral,2024-11-04,Jalna,2024-01-11,New,4,1819.01
1401,3665334102,435929,3,856.89,51476157,2024-11-04 01:16:31,5578.40,Toilet Cleaner,Household Care,Memon Ltd,...,7483634,3,Delivery,Neutral,2024-11-04,Burhanpur,2024-08-21,Inactive,10,1033.16
842,1816814476,667163,2,441.20,85198177,2024-11-04 13:50:21,4981.16,Orange Juice,Cold Drinks & Juices,Palla LLC,...,1558301,1,Delivery,Negative,2024-11-04,Jammu,2023-04-01,Premium,6,1666.01
